In [148]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the data

In [149]:
dataset = pd.read_csv('https://raw.githubusercontent.com/revanthmadasu/LifestyleWellbeing/master/Wellbeing_and_lifestyle_data_Kaggle.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

Converting categorial data into numerical

Understanding which objects to convert

In [150]:
object_dataset = dataset.select_dtypes(include=['object']).copy()
object_dataset.head()

,Timestamp,DAILY_STRESS,AGE,GENDER
0,7/7/15,2,36 to 50,Female
1,7/7/15,3,36 to 50,Female
2,7/7/15,3,36 to 50,Female
3,7/7/15,3,51 or more,Female
4,7/7/15,1,51 or more,Female


Converting Timestamp using label encoding.
"Finding and replacing" method would take forever.
"One-hot encoder" method would creat a lot more unnecessary columns


In [151]:
object_dataset['Timestamp'] = object_dataset['Timestamp'].astype('category')
object_dataset['Timestamp'] = object_dataset['Timestamp'].cat.codes
object_dataset.head()

,Timestamp,DAILY_STRESS,AGE,GENDER
0,5607,2,36 to 50,Female
1,5607,3,36 to 50,Female
2,5607,3,36 to 50,Female
3,5607,3,51 or more,Female
4,5607,1,51 or more,Female


Using "Finding and replacing" method, since there are values only from 0 to 5, with a arbitrary value of 1/1/2000 in row 10007

In [172]:
{try doing as a label encoder}#Daily_stress_nums = {'DAILY_STRESS':{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}}
object_dataset['DAILY_STRESS'] = object_dataset['DAILY_STRESS'].replace(['0','1','2','3','4','5','1/1/2000'], [0,1,2,3,4,5,1])
object_dataset.dtypes

Timestamp        int16
DAILY_STRESS    object
AGE             object
GENDER          object
dtype: object

Using get_dummies method for 'AGE' & 'GENDER' category

In [168]:
pd.get_dummies(object_dataset, columns = ['AGE', 'GENDER']).head()

,Timestamp,DAILY_STRESS,AGE_21 to 35,AGE_36 to 50,AGE_51 or more,AGE_Less than 20,GENDER_Female,GENDER_Male
0,5607,2,0,1,0,0,1,0
1,5607,3,0,1,0,0,1,0
2,5607,3,0,1,0,0,1,0
3,5607,3,0,0,1,0,1,0
4,5607,1,0,0,1,0,1,0


In [87]:
print(X[0,:])

['7/7/15' 3 '2' 2 5 0 5 2 0 1 6 4 5 0 7 5 5 1 4 0 5 '36 to 50' 'Female']


In [91]:
print(X)

[['7/7/15' 3 '2' ... 5 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 6 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 3 '36 to 50' 'Female']
 ...
 ['3/14/21 8:35' 4 '3' ... 10 '21 to 35' 'Male']
 ['3/14/21 8:43' 1 '1' ... 8 '21 to 35' 'Female']
 ['3/14/21 9:03' 5 '4' ... 4 '21 to 35' 'Female']]


In [89]:
print(y)

[609.5 655.6 631.6 ... 716.6 682.  651.4]


Handling missing data

In [ ]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer.fit(X[:, 1:3])
# X[:, 1:3] = imputer.transform(X[:, 1:3])